In [11]:
import numpy as np
import numpy.linalg
import scipy as sp
import scipy.stats
import matplotlib.pyplot as plt

%load_ext Cython


In [265]:
num_marks = 4
num_spikes = 100
num_pos_bins = 100
max_pos_bin = 20

marks = np.tile(np.array([np.arange(10,110,100/num_spikes)]).T, (1,num_marks))

pos_bins = np.arange(0, max_pos_bin, max_pos_bin/num_pos_bins)

pos_spikes = np.arange(0,pos_bins[-1], pos_bins[-1]/num_spikes)

pos_spikes = np.sin(pos_spikes) * max_pos_bin/2 +  max_pos_bin/2

mark_kernel_sd = 2
pos_kernel_sd = 2

In [266]:
%%prun -r -s cumulative
spike_pos_dist = sp.stats.norm.pdf(np.expand_dims(pos_bins, 0), 
                                   np.expand_dims(pos_spikes, 1),
                                   pos_kernel_sd)

dec_est = np.zeros([marks.shape[0], len(pos_bins)])
for dec_spk_ii, decode_mark in enumerate(marks):
    mark_contrib = np.prod(sp.stats.norm.pdf(decode_mark, marks, mark_kernel_sd), axis=1)
    dec_est[dec_spk_ii, :] = np.matmul(mark_contrib, spike_pos_dist)
    

In [267]:
plt.figure(figsize=[10,10])
plt.imshow(dec_est, extent=[pos_bins[0], pos_bins[-1], marks.shape[0], 0])
plt.show()

In [205]:
def multi_norm_pdf(x, mean, cov):
    x = np.array(x, ndmin=2)
    mean = np.array(mean, ndmin=2)
    cov = np.array(cov, ndmin=2)
    
    diff = x - mean
    prob = (1/np.sqrt(np.power((2 * np.pi), x.shape[-1]) * np.linalg.det(cov)) * 
            np.exp(-1/2 * np.matmul(np.matmul(diff, np.linalg.inv(cov)), diff.T)))
    if cov.ndim == 2:
        return np.diagonal(prob)
    elif cov.ndim == 3:
        return np.diagonal(np.diagonal(prob))


In [220]:
sp.stats.norm.pdf([[0,0,0,0]], loc=[[1,1,1,1], [2,2,2,2]], scale=[[1,1,1,1], [1,1,1,1]])

In [221]:
sp.stats.multivariate_normal.pdf([0,0,0,0], [2,2,2,2], np.eye(4))

In [207]:
multi_norm_pdf(np.array([[0, 0]]), np.array([[1, 1]]), np.eye(2))

In [193]:
np.array([[0,0]]).shape

In [24]:
x = np.array([1,1,1,1])
mean = np.random.randint(0,100,[20000,4])

#cov = [np.eye(4) for _ in range(mean.shape[0])]
cov = np.eye(4)

%time res = multi_norm_pdf(x, mean, cov)

In [18]:
%%time
val = []
for it in mean:
    val.append(scipy.stats.multivariate_normal.pdf([1,1,1,1],it,np.eye(4)))
